In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [2]:
train, test = pd.read_csv("/home/ryler/Datasets/Titanic-Starter-Competition/train.csv"), pd.read_csv("/home/ryler/Datasets/Titanic-Starter-Competition/test.csv")

In [3]:
def clean_data_for_titanic_training(dataset):
    data = dataset.drop(["PassengerId", "Embarked", "Parch", "SibSp", "Ticket", "Name", "Cabin"], axis=1)
    age_missing_values = data["Age"].isna()
    data.loc[age_missing_values, "Age"] = -1
    return data


In [4]:
cleaned_train = clean_data_for_titanic_training(train)
cleaned_test = clean_data_for_titanic_training(test)

cleaned_train

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500
...,...,...,...,...,...
886,0,2,male,27.0,13.0000
887,1,1,female,19.0,30.0000
888,0,3,female,-1.0,23.4500
889,1,1,male,26.0,30.0000


In [5]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.inference = nn.Sequential(
            nn.Linear(in_features=4, out_features=16),
            nn.BatchNorm1d(num_features=16),
            nn.ReLU(),
            nn.Linear(in_features=16, out_features=32),
            nn.BatchNorm1d(num_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=2),
        )

    def forward(self, inputs):
        return self.inference(inputs)

In [45]:
train_dataloader = DataLoader(cleaned_train, batch_size=4, shuffle=True)
features = next(iter(train_dataloader))

KeyError: 845